# ทดสอบ Face API - Thai HandMate

**การทดสอบ Face Expression Detection สำหรับโปรเจค Thai HandMate**

## เกี่ยวกับ Notebook นี้

ทดสอบ Face API ก่อนนำไปใช้ใน Frontend:
- ตรวจจับอารมณ์ (Happy, Sad, Angry, Surprised, Neutral, etc.)
- ตรวจจับใบหน้า (Face Detection)
- วัดความแม่นยำ (Confidence Score)

## ขั้นตอนการทำงาน

1. ติดตั้ง Libraries ที่จำเป็น
2. Download และตรวจสอบ Face Models  
3. ทดสอบ Model ด้วย Gradio Interface
4. Capture และสร้าง PNG 3 ไฟล์
5. ประมวลผล PNG ด้วย Model
6. แสดงผลเป็น JSON ในส่วนอารมณ์

In [ ]:
# Cell 1: ติดตั้งและ Import Libraries ที่จำเป็น
print("กำลังติดตั้ง libraries สำหรับ Face Detection...")

# ติดตั้ง packages ที่จำเป็น
import subprocess
import sys

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✓ ติดตั้ง {package} สำเร็จ")
    except Exception as e:
        print(f"✗ ติดตั้ง {package} ไม่สำเร็จ: {str(e)}")

# รายการ packages ที่จำเป็น
packages = [
    "opencv-python",
    "numpy", 
    "gradio",
    "Pillow",
    "face-recognition"  # สำหรับ face detection
]

for package in packages:
    install_package(package)

print("\n" + "="*50)
print("กำลัง Import Libraries...")

# Import libraries
import cv2
import numpy as np
import gradio as gr
from PIL import Image
import os
import json
import datetime

print("✓ OpenCV version:", cv2.__version__)
print("✓ NumPy version:", np.__version__)
print("✓ Gradio version:", gr.__version__)
print("✓ PIL version:", Image.__version__)

print("\n✅ ติดตั้งและ Import Libraries เสร็จสิ้น!")

In [ ]:
# Cell 2: Download และตรวจสอบ Face Models
print("กำลังตรวจสอบ Face Models...")

# กำหนด path ของ models
models_path = "."  # current directory
model_files = [
    "tiny_face_detector_model-weights_manifest.json",
    "tiny_face_detector_model-shard1",
    "face_landmark_68_model-weights_manifest.json", 
    "face_landmark_68_model-shard1",
    "face_expression_model-weights_manifest.json",
    "face_expression_model-shard1",
    "face_recognition_model-weights_manifest.json",
    "face_recognition_model-shard1",
    "ssd_mobilenetv1_model-weights_manifest.json"
]

# ตรวจสอบไฟล์ model ที่มีอยู่
print("\n📁 ตรวจสอบไฟล์ Face Models:")
print("="*60)

existing_models = []
missing_models = []

for model_file in model_files:
    file_path = os.path.join(models_path, model_file)
    if os.path.exists(file_path):
        file_size = os.path.getsize(file_path)
        size_mb = file_size / (1024 * 1024)
        existing_models.append(model_file)
        print(f"✓ {model_file} ({size_mb:.2f} MB)")
    else:
        missing_models.append(model_file)
        print(f"✗ {model_file} - ไฟล์หาไม่พบ")

print("\n📊 สรุปสถานะ Face Models:")
print("="*60)
print(f"✅ ไฟล์ที่พบ: {len(existing_models)} ไฟล์")
print(f"❌ ไฟล์ที่หาไม่พบ: {len(missing_models)} ไฟล์")

if existing_models:
    print("\n📋 รายการไฟล์ที่พร้อมใช้งาน:")
    for model in existing_models:
        print(f"  - {model}")

if missing_models:
    print("\n⚠️  รายการไฟล์ที่ต้องดาวน์โหลด:")
    for model in missing_models:
        print(f"  - {model}")
    print("\nแนะนำ: ดาวน์โหลดไฟล์จาก face-api.js models")
else:
    print("\n🎉 Face Models พร้อมใช้งานแล้ว!")

# ตรวจสอบ JavaScript runtime สำหรับ face-api.js (หากต้องการ)
print("\n💡 หมายเหตุ: การใช้ face-api.js ต้องมี JavaScript runtime")
print("💡 Notebook นี้จะใช้ OpenCV และ face-recognition library แทน")

In [ ]:
# Cell 3: ทดสอบ Face Detection ด้วย Gradio Interface
print("กำลังสร้าง Gradio Interface สำหรับทดสอบ Face Detection...")

import face_recognition

def detect_faces_and_emotions(image):
    """
    ฟังก์ชันสำหรับตรวจจับใบหน้า (ไม่มีการตรวจจับอารมณ์จริง)
    """
    try:
        # แปลง PIL Image เป็น numpy array
        image_array = np.array(image)
        
        # ตรวจจับใบหน้า
        face_locations = face_recognition.face_locations(image_array)
        
        # สร้างผลลัพธ์
        results = {
            "จำนวนใบหน้าที่พบ": len(face_locations),
            "ตำแหน่งใบหน้า": [],
            "สถานะ": "ตรวจจับใบหน้าเท่านั้น - ไม่มีโมเดลตรวจจับอารมณ์"
        }
        
        # วาดกรอบรอบใบหน้า
        image_with_faces = image_array.copy()
        
        for i, (top, right, bottom, left) in enumerate(face_locations):
            # วาดสี่เหลี่ยมรอบใบหน้า
            cv2.rectangle(image_with_faces, (left, top), (right, bottom), (0, 255, 0), 2)
            
            # เพิ่มข้อมูลตำแหน่ง
            results["ตำแหน่งใบหน้า"].append({
                "ใบหน้าที่": i + 1,
                "ตำแหน่ง": f"({left}, {top}) ถึง ({right}, {bottom})",
                "ขนาด": f"{right-left} x {bottom-top} pixels"
            })
            
            # เขียนข้อความบนภาพ
            cv2.putText(image_with_faces, f"Face {i+1} - No Emotion Model", 
                       (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        if len(face_locations) == 0:
            results["สถานะ"] = "ไม่พบใบหน้าในภาพนี้"
        
        return image_with_faces, json.dumps(results, ensure_ascii=False, indent=2)
        
    except Exception as e:
        error_msg = f"เกิดข้อผิดพลาด: {str(e)}"
        return image, error_msg

# สร้าง Gradio Interface
print("🎯 สร้าง Gradio Interface...")

demo = gr.Interface(
    fn=detect_faces_and_emotions,
    inputs=[
        gr.Image(type="pil", label="อัปโหลดรูปภาพ")
    ],
    outputs=[
        gr.Image(type="numpy", label="ผลลัพธ์การตรวจจับใบหน้า"),
        gr.Textbox(label="ข้อมูลการตรวจจับ (JSON)", lines=10)
    ],
    title="🔍 ทดสอบ Face Detection - Thai HandMate",
    description="อัปโหลดรูปภาพเพื่อทดสอบการตรวจจับใบหน้า (ไม่มีการตรวจจับอารมณ์)",
    examples=None
)

print("✅ Gradio Interface พร้อมใช้งาน!")
print("💡 หมายเหตุ: Interface นี้จะตรวจจับใบหน้าเท่านั้น ไม่มีการตรวจจับอารมณ์")
print("💡 กด 'Launch' เพื่อเปิดใช้งาน Interface")

# เปิด Gradio Interface
demo.launch(share=False, debug=True)

In [ ]:
# Cell 4: Capture และสร้าง PNG 3 ไฟล์
print("กำลังเปิดกล้องสำหรับ Capture ภาพ...")

import time
from datetime import datetime

class CameraCapture:
    def __init__(self):
        self.cap = None
        self.captured_images = []
        
    def initialize_camera(self):
        """เปิดกล้อง"""
        try:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                print("❌ ไม่สามารถเปิดกล้องได้")
                return False
            
            # ตั้งค่าความละเอียด
            self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
            self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
            
            print("✅ เปิดกล้องสำเร็จ")
            return True
        except Exception as e:
            print(f"❌ เกิดข้อผิดพลาดในการเปิดกล้อง: {str(e)}")
            return False
    
    def capture_images(self, num_images=3):
        """ถ่ายภาพตามจำนวนที่กำหนด"""
        if not self.initialize_camera():
            return []
        
        self.captured_images = []
        print(f"\n📸 เริ่มถ่ายภาพ {num_images} รูป")
        print("💡 กด 'c' เพื่อถ่ายภาพ, 'q' เพื่อออก")
        print("="*50)
        
        capture_count = 0
        
        while capture_count < num_images:
            ret, frame = self.cap.read()
            if not ret:
                print("❌ ไม่สามารถอ่านข้อมูลจากกล้องได้")
                break
            
            # แสดงภาพปัจจุบัน
            display_frame = frame.copy()
            
            # เขียนข้อความบนภาพ
            cv2.putText(display_frame, f"Image {capture_count + 1}/{num_images}", 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(display_frame, "Press 'c' to capture, 'q' to quit", 
                       (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            
            cv2.imshow('Camera - Face Detection Test', display_frame)
            
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('c'):  # กด 'c' เพื่อถ่ายภาพ
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"face_test_image_{capture_count + 1}_{timestamp}.png"
                
                # บันทึกภาพ
                cv2.imwrite(filename, frame)
                self.captured_images.append(filename)
                capture_count += 1
                
                print(f"📸 ถ่ายภาพ {capture_count}: {filename}")
                
                # แสดงภาพที่ถ่าย
                cv2.imshow(f'Captured Image {capture_count}', frame)
                time.sleep(0.5)  # หน่วงเวลาเล็กน้อย
                
            elif key == ord('q'):  # กด 'q' เพื่อออก
                print("🚫 ยกเลิกการถ่ายภาพ")
                break
        
        # ปิดกล้องและหน้าต่าง
        self.cap.release()
        cv2.destroyAllWindows()
        
        return self.captured_images
    
    def auto_capture(self, num_images=3, delay=3):
        """ถ่ายภาพอัตโนมัติ"""
        if not self.initialize_camera():
            return []
        
        self.captured_images = []
        print(f"\n📸 ถ่ายภาพอัตโนมัติ {num_images} รูป (ห่าง {delay} วินาที)")
        print("="*50)
        
        for i in range(num_images):
            print(f"⏳ เตรียมถ่ายภาพที่ {i + 1} ใน {delay} วินาที...")
            
            # นับถอยหลัง
            for countdown in range(delay, 0, -1):
                ret, frame = self.cap.read()
                if ret:
                    display_frame = frame.copy()
                    cv2.putText(display_frame, f"Image {i + 1}/{num_images}", 
                               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv2.putText(display_frame, f"Capturing in: {countdown}", 
                               (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    cv2.imshow('Auto Capture - Face Detection Test', display_frame)
                    
                if cv2.waitKey(1000) & 0xFF == ord('q'):
                    self.cap.release()
                    cv2.destroyAllWindows()
                    return self.captured_images
            
            # ถ่ายภาพ
            ret, frame = self.cap.read()
            if ret:
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"face_test_image_{i + 1}_{timestamp}.png"
                
                cv2.imwrite(filename, frame)
                self.captured_images.append(filename)
                print(f"📸 ถ่ายภาพ {i + 1}: {filename}")
        
        self.cap.release()
        cv2.destroyAllWindows()
        return self.captured_images

# สร้าง instance และเริ่มถ่ายภาพ
camera = CameraCapture()

print("🎯 เลือกโหมดการถ่ายภาพ:")
print("1. Manual Capture (กด 'c' เพื่อถ่าย)")
print("2. Auto Capture (ถ่ายอัตโนมัติทุก 3 วินาที)")

# ใช้ auto capture เป็นค่าเริ่มต้น
print("\n🤖 ใช้โหมด Auto Capture...")
captured_files = camera.auto_capture(num_images=3, delay=3)

print(f"\n✅ ถ่ายภาพเสร็จสิ้น! ได้ภาพทั้งหมด: {len(captured_files)} ไฟล์")
print("📁 รายการไฟล์ที่ถ่าย:")
for i, filename in enumerate(captured_files, 1):
    file_size = os.path.getsize(filename) / 1024  # KB
    print(f"  {i}. {filename} ({file_size:.1f} KB)")

In [ ]:
# Cell 5: ประมวลผล PNG ด้วย Face Detection Model
print("กำลังประมวลผลภาพด้วย Face Detection Model...")

import glob
import matplotlib.pyplot as plt

class FaceProcessor:
    def __init__(self):
        self.results = []
    
    def process_image(self, image_path):
        """ประมวลผลภาพเดี่ยว - ตรวจจับใบหน้าเท่านั้น"""
        try:
            # อ่านภาพ
            image = cv2.imread(image_path)
            if image is None:
                return {"error": f"ไม่สามารถอ่านไฟล์ {image_path}"}
            
            # แปลงสีจาก BGR เป็น RGB
            rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # ตรวจจับใบหน้า
            face_locations = face_recognition.face_locations(rgb_image)
            
            # สร้างผลลัพธ์
            result = {
                "ไฟล์": os.path.basename(image_path),
                "จำนวนใบหน้า": len(face_locations),
                "ขนาดภาพ": f"{image.shape[1]} x {image.shape[0]} pixels",
                "ใบหน้าที่พบ": [],
                "สถานะอารมณ์": "ไม่มีโมเดลตรวจจับอารมณ์"
            }
            
            # วิเคราะห์แต่ละใบหน้า
            for i, (top, right, bottom, left) in enumerate(face_locations):
                face_info = {
                    "ใบหน้าที่": i + 1,
                    "ตำแหน่ง": {
                        "left": int(left),
                        "top": int(top), 
                        "right": int(right),
                        "bottom": int(bottom)
                    },
                    "ขนาดใบหน้า": f"{right-left} x {bottom-top} pixels",
                    "อารมณ์หลัก": "ไม่พบ",
                    "ความมั่นใจ": 0.0,
                    "หมายเหตุ": "ไม่มีโมเดลตรวจจับอารมณ์"
                }
                
                result["ใบหน้าที่พบ"].append(face_info)
                
                # วาดกรอบบนภาพ
                cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.putText(image, f"Face {i+1} - No Emotion", 
                           (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            
            # บันทึกภาพที่มีกรอบ
            if face_locations:
                output_path = image_path.replace('.png', '_processed.png')
                cv2.imwrite(output_path, image)
                result["ภาพผลลัพธ์"] = output_path
            else:
                result["สถานะ"] = "ไม่พบใบหน้า - ไม่สร้างภาพผลลัพธ์"
            
            return result
            
        except Exception as e:
            return {"error": f"เกิดข้อผิดพลาด: {str(e)}"}
    
    def process_all_images(self):
        """ประมวลผลภาพทั้งหมด"""
        # หาไฟล์ PNG ที่ถ่ายไว้
        image_files = glob.glob("face_test_image_*.png")
        image_files = [f for f in image_files if '_processed' not in f]  # ไม่เอาไฟล์ processed
        
        if not image_files:
            print("❌ ไม่พบไฟล์ภาพที่ถ่ายไว้")
            print("💡 กรุณารันเซลล์ที่ 4 เพื่อถ่ายภาพก่อน")
            return []
        
        print(f"📸 พบภาพที่ถ่ายไว้: {len(image_files)} ไฟล์")
        print("="*60)
        
        self.results = []
        
        for image_file in sorted(image_files):
            print(f"\n🔍 กำลังประมวลผล: {image_file}")
            result = self.process_image(image_file)
            
            if "error" in result:
                print(f"❌ {result['error']}")
            else:
                self.results.append(result)
                print(f"✅ พบใบหน้า: {result['จำนวนใบหน้า']} ใบหน้า")
                
                # แสดงรายละเอียดใบหน้า
                if result['จำนวนใบหน้า'] > 0:
                    for face in result["ใบหน้าที่พบ"]:
                        print(f"   - ใบหน้าที่ {face['ใบหน้าที่']}: ไม่มีการตรวจจับอารมณ์")
                else:
                    print("   - ไม่พบใบหน้าในภาพนี้")
        
        return self.results
    
    def display_summary(self):
        """แสดงสรุปผลการประมวลผล"""
        if not self.results:
            print("❌ ไม่มีผลลัพธ์ให้แสดง")
            return
        
        print("\n" + "="*60)
        print("📊 สรุปผลการตรวจจับใบหน้า")
        print("="*60)
        
        total_faces = sum(result["จำนวนใบหน้า"] for result in self.results)
        total_images = len(self.results)
        
        print(f"📸 ภาพทั้งหมด: {total_images} ไฟล์")
        print(f"👥 ใบหน้าทั้งหมด: {total_faces} ใบหน้า")
        print(f"📈 ค่าเฉลี่ย: {total_faces/total_images:.1f} ใบหน้าต่อภาพ")
        print(f"🎭 สถานะอารมณ์: ไม่มีโมเดลตรวจจับอารมณ์")

# สร้าง processor และเริ่มประมวลผล
processor = FaceProcessor()
results = processor.process_all_images()
processor.display_summary()

print(f"\n💾 เก็บผลลัพธ์ไว้ในตัวแปร 'results' สำหรับใช้ในเซลล์ถัดไป")
print(f"📁 ผลลัพธ์: {len(results)} ไฟล์")
print(f"💡 หมายเหตุ: ผลลัพธ์จะแสดงเฉพาะการตรวจจับใบหน้า ไม่มีข้อมูลอารมณ์")

In [ ]:
# Cell 6: แสดงผล JSON ในส่วนอารมณ์
print("กำลังสร้าง JSON Output สำหรับส่วนอารมณ์...")

def create_emotion_json(results):
    """สร้าง JSON ที่แสดงสถานะไม่มีข้อมูลอารมณ์"""
    if not results:
        return {"error": "ไม่มีผลลัพธ์ให้แสดง"}
    
    emotion_data = {
        "timestamp": datetime.now().isoformat(),
        "total_images": len(results),
        "total_faces": sum(result["จำนวนใบหน้า"] for result in results),
        "emotion_status": "ไม่มีโมเดลตรวจจับอารมณ์",
        "images": []
    }
    
    for result in results:
        image_data = {
            "filename": result["ไฟล์"],
            "faces_detected": result["จำนวนใบหน้า"],
            "faces": []
        }
        
        for face in result["ใบหน้าที่พบ"]:
            face_emotion = {
                "face_id": face["ใบหน้าที่"],
                "primary_emotion": "ไม่พบ",
                "confidence": 0.0,
                "emotion_scores": "ไม่มีข้อมูล",
                "face_position": face["ตำแหน่ง"],
                "status": "ไม่มีโมเดลตรวจจับอารมณ์"
            }
            image_data["faces"].append(face_emotion)
        
        emotion_data["images"].append(image_data)
    
    return emotion_data

def create_summary_json(results):
    """สร้าง JSON สรุปที่แสดงสถานะไม่มีข้อมูลอารมณ์"""
    if not results:
        return {"error": "ไม่มีผลลัพธ์ให้แสดง"}
    
    total_faces = sum(result["จำนวนใบหน้า"] for result in results)
    
    summary_data = {
        "timestamp": datetime.now().isoformat(),
        "summary": {
            "total_images_processed": len(results),
            "total_faces_detected": total_faces,
            "average_faces_per_image": round(total_faces / len(results), 2),
            "emotion_detection_status": "ไม่มีโมเดลตรวจจับอารมณ์"
        },
        "emotion_distribution": "ไม่มีข้อมูล",
        "message": "สามารถตรวจจับใบหน้าได้ แต่ไม่มีการตรวจจับอารมณ์"
    }
    
    return summary_data

# ตรวจสอบว่ามีผลลัพธ์หรือไม่
if 'results' in locals() and results:
    print("✅ พบผลลัพธ์จากเซลล์ก่อนหน้า")
    
    # สร้าง JSON รายละเอียด
    print("\n📄 JSON รายละเอียดทั้งหมด:")
    print("="*60)
    emotion_json = create_emotion_json(results)
    print(json.dumps(emotion_json, ensure_ascii=False, indent=2))
    
    # สร้าง JSON สรุป
    print("\n📊 JSON สรุปการตรวจจับ:")
    print("="*60)
    summary_json = create_summary_json(results)
    print(json.dumps(summary_json, ensure_ascii=False, indent=2))
    
    # บันทึกไฟล์ JSON
    with open("face_detection_results.json", "w", encoding="utf-8") as f:
        json.dump(emotion_json, f, ensure_ascii=False, indent=2)
    
    with open("face_detection_summary.json", "w", encoding="utf-8") as f:
        json.dump(summary_json, f, ensure_ascii=False, indent=2)
    
    print(f"\n💾 บันทึกไฟล์ JSON แล้ว:")
    print(f"  - face_detection_results.json ({os.path.getsize('face_detection_results.json')} bytes)")
    print(f"  - face_detection_summary.json ({os.path.getsize('face_detection_summary.json')} bytes)")
    
    # แสดงสถานะไม่มีอารมณ์
    print(f"\n🎭 สถานะการตรวจจับอารมณ์:")
    print("="*60)
    simple_status = {}
    for img_data in emotion_json["images"]:
        filename = img_data["filename"]
        face_count = img_data["faces_detected"]
        simple_status[filename] = {
            "faces_detected": face_count,
            "emotion_status": "ไม่มีโมเดลตรวจจับอารมณ์"
        }
    
    print(json.dumps(simple_status, ensure_ascii=False, indent=2))

else:
    print("❌ ไม่พบผลลัพธ์จากเซลล์ก่อนหน้า")
    print("💡 กรุณารันเซลล์ที่ 5 เพื่อประมวลผลภาพก่อน")
    
    # สร้าง JSON ตัวอย่างที่แสดงสถานะไม่มีอารมณ์
    sample_json = {
        "error": "ไม่มีข้อมูล",
        "message": "กรุณารันเซลล์การประมวลผลภาพก่อน",
        "expected_format": {
            "timestamp": "2025-09-13T12:00:00",
            "emotion_status": "ไม่มีโมเดลตรวจจับอารมณ์",
            "images": [
                {
                    "filename": "face_test_image_1.png",
                    "faces_detected": 1,
                    "faces": [
                        {
                            "face_id": 1,
                            "primary_emotion": "ไม่พบ",
                            "confidence": 0.0,
                            "emotion_scores": "ไม่มีข้อมูล",
                            "status": "ไม่มีโมเดลตรวจจับอารมณ์"
                        }
                    ]
                }
            ]
        }
    }
    
    print("\n📄 รูปแบบ JSON ที่คาดหวัง:")
    print(json.dumps(sample_json, ensure_ascii=False, indent=2))

print(f"\n✅ เสร็จสิ้นการแสดงผล JSON สำหรับ Face Detection!")
print(f"💡 หมายเหตุ: ไม่มีการใช้ mock data - แสดงสถานะจริงเท่านั้น")